# Exploring the CommaAI Device data

This requires python 3.11 or later because of the usage of the tools folder from Openpilot, which uses 3.11 features

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import pandas as pd
from pathlib import Path
# This is taken from the Openpilot project
from openpilot.tools.lib.logreader import LogReader, ReadMode
from dataclasses import dataclass
from datetime import datetime
from typing import List
import capnp
import json

In [2]:
# device_path = Path("/home/ulrikro/datasets/CommaAI/2024_01_14_TrondheimSnow/2024-01-14--13-01-26--10/")  # Old
device_path = Path("/home/ulrikro/datasets/CommaAI/2024_02_28_Orkdal/2024-02-28--10-54-05--22/")  # New

# About the folder

Each folder, marked by their data and a number, contains 5 files: ecamera.hevc, fcamera.hevc, qcamera.ts, qlog, rlog. Each file contains 1 min video. 

* ecamera.hevc - road facing wide angle camera 1928*1208 at 20Hz
* fcamera.hevc - road facing narrow camera 1928*1208 at 20Hz
* qcamera.ts - driver facing wide angle camera 526*330 at 20Hz
* qlog - some
* rlog - some



In [3]:
ecamera_path = device_path / "ecamera.hevc"
fcamera_path = device_path / "fcamera.hevc"
qcamera_path = device_path / "qcamera.ts"
qlog_path = device_path / "qlog"
rlog_path = device_path / "rlog"


# Overview of the video files

In [4]:
def read_video_frames(file_path: str):
    frames = []
    video = cv2.VideoCapture(file_path)
    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break
        frames.append(frame)
    video.release()
    return frames

In [5]:
# ecamera_frames = read_video_frames(ecamera_path.as_posix())
# len(ecamera_frames)

In [6]:
# ecamera_frames[0].shape

In [7]:
# fcamera_frames = read_video_frames(fcamera_path.as_posix())
#len(fcamera_frames)

In [8]:
# fcamera_frames[0].shape

In [9]:
# qcamera_frames = read_video_frames(qcamera_path.as_posix())
# len(qcamera_frames)

In [10]:
# qcamera_frames[0].shape

# Overview of the log files

In [11]:


qlog_data = LogReader(qlog_path.as_posix(), default_mode=ReadMode.QLOG)

In [12]:
qlog_list = list(qlog_data)
len(qlog_list)

11363

In [13]:
type(qlog_list[0])

capnp.lib.capnp._DynamicStructReader

Each iteration throug the log data retruns a capnp.lib.capnp._DynamicStructReader object. 

capnp is a proto schema parsing library 

The proto schema seems to be defined in openpilot/common/params.cc
 


In [14]:
@dataclass
class CommandEntry:
    key: str
    value: str

@dataclass
class Commands:
    entries: List[CommandEntry]

@dataclass
class ParamEntry:
    key: str
    value: str

@dataclass
class Params:
    entries: List[ParamEntry]

@dataclass
class InitData:
    kernelArgs: List[str]
    dongleId: str
    deviceType: str
    version: str
    dirty: bool
    gitCommit: str
    gitBranch: str
    passive: bool
    gitRemote: str
    kernelVersion: str
    params: Params
    osVersion: str
    commands: Commands

@dataclass
class LogData:
    logMonoTime: int
    initData: InitData
    valid: bool
    # these just include a subset of the data

In [15]:
qlog_list: List[LogData] = list(qlog_data)

In [16]:
print([attr for attr in dir(qlog_list[0]) if not attr.startswith("__")])

['_get', '_get_by_field', '_has', '_has_by_field', '_parent', '_which', '_which_str', 'accelerometer', 'accelerometer2', 'androidGnssDEPRECATED', 'androidLog', 'applanixLocationDEPRECATED', 'applanixRawDEPRECATED', 'as_builder', 'boot', 'cameraOdometry', 'can', 'carControl', 'carOutput', 'carParams', 'carState', 'cellInfoDEPRECATED', 'clocks', 'controlsState', 'customReserved0', 'customReserved1', 'customReserved2', 'customReserved3', 'customReserved4', 'customReserved5', 'customReserved6', 'customReserved7', 'customReserved8', 'customReserved9', 'customReservedRawData0', 'customReservedRawData1', 'customReservedRawData2', 'deviceState', 'driverCameraState', 'driverEncodeData', 'driverEncodeIdx', 'driverMonitoringState', 'driverStateDEPRECATED', 'driverStateV2', 'errorLogMessage', 'ethernetDataDEPRECATED', 'featuresDEPRECATED', 'gnssMeasurements', 'gpsLocation', 'gpsLocationExternal', 'gpsNMEA', 'gpsPlannerPlanDEPRECATED', 'gpsPlannerPointsDEPRECATED', 'gyroscope', 'gyroscope2', 'initD

In [17]:
print(str(qlog_list[0].initData.params.entries[0].value))

b'{"alias": null, "athena_host": "prod-comma-public-athena-3.prod-comma-public-athena.production.svc.cluster.local", "device_type": "threex", "dongle_id": "b1f2aa9141431844", "ignore_uploads": null, "last_athena_ping": 1708958612, "last_gps_accuracy": null, "last_gps_bearing": null, "last_gps_lat": null, "last_gps_lng": null, "last_gps_speed": null, "last_gps_time": null, "public_key": "-----BEGIN PUBLIC KEY-----\\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA0S8Afqnskw9zXQsgrDZl\\nsZpcKY2dQrt3SP01F2K19SmjJdpjFm0c3+U/0sDi3MNSZD5EKWj2zSTGynsiITW2\\n+cuT/IdeCpLygoBWQ1uYBN7iiq5fa0fNpkRrrK3Aetgm+H1nu/W1/dSRXg0EDvQo\\n9+AvU9OBjGxJYd0ew+pt2MxLXUJ5ffRiNRWmKBaAsrQoYnS9gtrrncKADsYbkh3E\\ncpPvX+L5Rkpoi/iFFfyqZYmO25BAuZ1ACsX9aEvBQ3DvOjilC4giJO42/Y/LSxx0\\nhG4dmyiucmOFNmPPlZAxeKWdeBZrz7S8AsecKxoBUOg4B/ipfBTSho7l3Fx0Kgow\\nuwIDAQAB\\n-----END PUBLIC KEY-----\\n", "serial": "c6a3e52b", "sim_id": "", "trial_claimed": false, "is_paired": true, "eligible_features": {"prime": true, "prime_data": true, "n

In [18]:
qlog_list[0].to_dict()

{'initData': {'kernelArgs': ['rcupdate.rcu_expedited=1',
   'console=ttyMSM0,115200n8',
   'quiet',
   'loglevel=3',
   'earlycon=msm_geni_serial,0xA84000',
   'androidboot.hardware=qcom',
   'androidboot.console=ttyMSM0',
   'video=DSI-1:1080x2160@60e',
   'ehci-hcd.park=3',
   'lpm_levels.sleep_disabled=1',
   'service_locator.enable=1',
   'androidboot.selinux=permissive',
   'firmware_class.path=/lib/firmware/updates',
   'net.ifnames=0',
   'dyndbg=""',
   'root=/dev/sda6',
   'androidboot.bootdevice=1d84000.ufshc',
   'androidboot.serialno=c6a3e52b',
   'androidboot.baseband=sda',
   'msm_drm.dsi_display0=dsi_fhd_ea8074_1080_cmd_display:',
   'androidboot.slot_suffix=_a',
   'skip_initramfs',
   'rootwait',
   'ro',
   'init=/sbin/init'],
  'dongleId': 'b1f2aa9141431844',
  'deviceType': 'tizi',
  'version': '0.9.7',
  'dirty': False,
  'gitCommit': '8e03cdfc2ae72ea09208ce41ddc8f7a118c3b973',
  'gitBranch': 'master',
  'passive': False,
  'gitRemote': 'https://github.com/commaai/

In [19]:
def isiterable(item):
    try:
        iter(item)
        return True
    except TypeError:
        return False
def get_fields(o, print_progress=True, depth=0):
    if isinstance(o, list) or isinstance(o, tuple) or isinstance(o, set) or isiterable(o):
        fields = []
        for item in o:
            field = get_fields(item, print_progress, depth=depth+1)
            fields.append(field)
        return fields
    elif isinstance(o, dict):
        total_dict = dict()
        for key, value in o.items():
            total_dict[key] = get_fields(value, print_progress, depth=depth+1)
        return total_dict
    elif isinstance(o, str) or isinstance(o, int) or isinstance(o, float) or isinstance(o, bool) or isinstance(o, bytes) or o is None:
        return type(o).__name__
    else:
        item_attrs = [attr for attr in dir(o) if not attr.startswith("_")]
        # print("item_attrs", item_attrs)
        total_dict = dict()
        progress = 0
        for attr in item_attrs:
            if print_progress:
                progress += 1
                print(f"Progress {depth}: {progress}/{len(item_attrs)} attr={attr}", end="\r")
            try:
                total_dict[attr] = get_fields(getattr(o, attr), print_progress, depth=depth+1)
            except Exception:
                continue
        return total_dict



In [20]:
import re

def parse_capnp_string(s: str):
    # Wrap words after = that are not numbers or true of false in quotes
    s = re.sub(r'(?<=\s=\s)(?!true|false|\d+)(\w+)', r'"\1"', s)
    # Wrap in quotes and replace = with :
    s = re.sub(r"(\w+) = ", r'"\1": ', s)

    # Replace ( with [ for lists
    pattern = r'\((?:"[^"]*"|\d+)(?:\s*,\s*(?:"[^"]*"|\d+))*\)'

    def convert(match):
        inner_text = match.group(0)[1:-1]  # Remove the outer parentheses
        return "[" + inner_text + "]"  # Surround with square brackets

    s = re.sub(pattern, convert, s)

    # Replace the rest of the parentheses with curly braces and remove newlines
    s = s.replace("(", "{").replace(")", "}").replace("\n", "")
    return s

In [21]:
# import json
# field_json = capnp_to_json(str(qlog_list[0]))
# field_json
# parse_custom_structure(str(qlog_list[0]))

In [22]:
qlog_list[0].to_dict()

{'initData': {'kernelArgs': ['rcupdate.rcu_expedited=1',
   'console=ttyMSM0,115200n8',
   'quiet',
   'loglevel=3',
   'earlycon=msm_geni_serial,0xA84000',
   'androidboot.hardware=qcom',
   'androidboot.console=ttyMSM0',
   'video=DSI-1:1080x2160@60e',
   'ehci-hcd.park=3',
   'lpm_levels.sleep_disabled=1',
   'service_locator.enable=1',
   'androidboot.selinux=permissive',
   'firmware_class.path=/lib/firmware/updates',
   'net.ifnames=0',
   'dyndbg=""',
   'root=/dev/sda6',
   'androidboot.bootdevice=1d84000.ufshc',
   'androidboot.serialno=c6a3e52b',
   'androidboot.baseband=sda',
   'msm_drm.dsi_display0=dsi_fhd_ea8074_1080_cmd_display:',
   'androidboot.slot_suffix=_a',
   'skip_initramfs',
   'rootwait',
   'ro',
   'init=/sbin/init'],
  'dongleId': 'b1f2aa9141431844',
  'deviceType': 'tizi',
  'version': '0.9.7',
  'dirty': False,
  'gitCommit': '8e03cdfc2ae72ea09208ce41ddc8f7a118c3b973',
  'gitBranch': 'master',
  'passive': False,
  'gitRemote': 'https://github.com/commaai/

In [23]:
import base64
class BytesEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, bytes):
            # Encode bytes to a Base64 string
            return base64.b64encode(obj).decode('ascii')
        # Let the base class default method raise the TypeError
        return json.JSONEncoder.default(self, obj)

def capnp_to_json_valid_dict(obj):
    json_string = json.dumps(obj.to_dict(), cls=BytesEncoder)
    return json.loads(json_string)


In [24]:
qlog_dicts = [capnp_to_json_valid_dict(o) for o in qlog_list]

In [25]:
# jsons = [json.dumps(qlog_dict, indent=4) for qlog_dict in qlog_dicts]
qlog_name = "qlog" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + ".json"
with open(qlog_name, "w") as f:
    json.dump(qlog_dicts, f, indent=4)

In [26]:
for field in list(qlog_list[0].schema.node.struct.fields):
    print(field.name)

logMonoTime
initData
roadCameraState
gpsNMEA
sensorEventDEPRECATED
can
deviceState
controlsState
liveEventDEPRECATED
model
featuresDEPRECATED
sensorEventsDEPRECATED
pandaStateDEPRECATED
radarState
liveUIDEPRECATED
roadEncodeIdx
liveTracks
sendcan
logMessage
liveCalibration
androidLog
gpsLocation
carState
carControl
longitudinalPlan
liveLocationDEPRECATED
ethernetDataDEPRECATED
navUpdateDEPRECATED
cellInfoDEPRECATED
wifiScanDEPRECATED
androidGnssDEPRECATED
qcomGnss
lidarPtsDEPRECATED
procLog
ubloxGnss
clocks
liveMpcDEPRECATED
liveLongitudinalMpcDEPRECATED
navStatusDEPRECATED
ubloxRaw
gpsPlannerPointsDEPRECATED
gpsPlannerPlanDEPRECATED
applanixRawDEPRECATED
trafficEventsDEPRECATED
liveLocationTimingDEPRECATED
orbslamCorrectionDEPRECATED
liveLocationCorrectedDEPRECATED
orbObservationDEPRECATED
gpsLocationExternal
locationDEPRECATED
uiNavigationEventDEPRECATED
liveLocationKalmanDEPRECATED
testJoystick
orbOdometryDEPRECATED
orbFeaturesDEPRECATED
applanixLocationDEPRECATED
orbKeyFrameDEPRECA

In [27]:
result = parse_capnp_string(str(qlog_list[0]))
result

'{ "logMonoTime": 267068185730,  "initData": {    "kernelArgs": [      "rcupdate.rcu_expedited=1",      "console=ttyMSM0,115200n8",      "quiet",      "loglevel=3",      "earlycon=msm_geni_serial,0xA84000",      "androidboot.hardware=qcom",      "androidboot.console=ttyMSM0",      "video=DSI-1:1080x2160@60e",      "ehci-hcd.park=3",      "lpm_levels.sleep_disabled=1",      "service_locator.enable=1",      "androidboot.selinux=permissive",      "firmware_class.path=/lib/firmware/updates",      "net.ifnames=0",      "dyndbg=\\"\\"",      "root=/dev/sda6",      "androidboot.bootdevice=1d84000.ufshc",      "androidboot.serialno=c6a3e52b",      "androidboot.baseband=sda",      "msm_drm.dsi_display0=dsi_fhd_ea8074_1080_cmd_display:",      "androidboot.slot_suffix=_a",      "skip_initramfs",      "rootwait",      "ro",      "init=/sbin/init" ],    "dongleId": "b1f2aa9141431844",    "deviceType": "tizi",    "version": "0.9.7",    "dirty": false,    "gitCommit": "8e03cdfc2ae72ea09208ce41ddc8f7a

In [28]:
import json
#json.loads(result)

In [29]:
model_qlogs = [log for log in qlog_list if log.which() == "modelV2"]
len(model_qlogs)

30

In [30]:
print(model_qlogs[3])

( logMonoTime = 1595427581264,
  valid = true,
  modelV2 = (
    frameId = 26539,
    frameAge = 0,
    frameDropPerc = 0,
    timestampEof = 1595391426160,
    position = (
      x = [-0.37291586, -0.41735703, 0.36903945, 1.4470326, 2.9558794, 4.8989267, 7.2574816, 10.118219, 13.346531, 17.038332, 21.175682, 25.672699, 30.662745, 36.114632, 42.0127, 48.231468, 54.938896, 61.988361, 69.567764, 77.33461, 85.82164, 94.577164, 104.08485, 113.49332, 123.47732, 134.17041, 144.59047, 155.6521, 167.38173, 178.893, 191.70004, 204.47479, 216.78218],
      y = [0.0026716292, 0.0011429004, 0.0092234761, 0.0094248988, 0.012509517, 0.025470816, 0.034656394, 0.068762109, 0.1068406, 0.1662669, 0.24243164, 0.36979342, 0.50235021, 0.69327557, 0.92546409, 1.2345074, 1.5873841, 2.0435472, 2.574918, 3.1920619, 3.9295578, 4.7882366, 5.8132915, 6.95158, 8.2174435, 9.6846409, 11.331856, 13.133395, 15.160534, 17.369598, 19.748413, 22.42289, 25.222322],
      z = [-0.001636903, -0.0021418352, -0.0036602104, -0

# The different types of CAN messages

In [31]:
types = [item.which() for item in qlog_list]
len(types)

11363

In [32]:
qlog_types = set(types)
len(qlog_types)

43

In [33]:
qlog_types

{'accelerometer',
 'cameraOdometry',
 'can',
 'carControl',
 'carParams',
 'carState',
 'clocks',
 'controlsState',
 'deviceState',
 'driverCameraState',
 'driverEncodeIdx',
 'driverMonitoringState',
 'driverStateV2',
 'gpsLocation',
 'gyroscope',
 'initData',
 'liveCalibration',
 'liveLocationKalman',
 'liveParameters',
 'liveTorqueParameters',
 'longitudinalPlan',
 'magnetometer',
 'managerState',
 'mapRenderState',
 'microphone',
 'modelV2',
 'navInstruction',
 'navModelDEPRECATED',
 'onroadEvents',
 'pandaStates',
 'peripheralState',
 'procLog',
 'qRoadEncodeIdx',
 'radarState',
 'roadCameraState',
 'roadEncodeIdx',
 'sendcan',
 'sentinel',
 'thumbnail',
 'uiDebug',
 'uiPlan',
 'wideRoadCameraState',
 'wideRoadEncodeIdx'}

Which of these are the most interesting to look at? 


1. modelV2


# RLOG



In [34]:
rlog_data = LogReader(rlog_path.as_posix(), default_mode=ReadMode.RLOG)
rlog_list = list(rlog_data)
rlog_types = set([item.which() for item in rlog_list])

In [35]:
len(qlog_types), len(rlog_types)

(43, 49)

In [36]:
qlog_types == rlog_types

False

In [37]:
len(rlog_list)

69346

In [38]:
# Create rlog json file
from datetime import datetime
rlog_dicts = [capnp_to_json_valid_dict(o) for o in rlog_list]
name = "rlog" + datetime.now().strftime("%Y%m%d-%H%M%S") + ".json"
with open(name, "w") as f:
    json.dump(rlog_dicts, f, indent=4)

In [39]:
len(rlog_types)

49

In [40]:
rlog_types - qlog_types

{'androidLog',
 'liveTracks',
 'logMessage',
 'navThumbnail',
 'qcomGnss',
 'temperatureSensor'}

In [41]:
rlog_types

{'accelerometer',
 'androidLog',
 'cameraOdometry',
 'can',
 'carControl',
 'carParams',
 'carState',
 'clocks',
 'controlsState',
 'deviceState',
 'driverCameraState',
 'driverEncodeIdx',
 'driverMonitoringState',
 'driverStateV2',
 'gpsLocation',
 'gyroscope',
 'initData',
 'liveCalibration',
 'liveLocationKalman',
 'liveParameters',
 'liveTorqueParameters',
 'liveTracks',
 'logMessage',
 'longitudinalPlan',
 'magnetometer',
 'managerState',
 'mapRenderState',
 'microphone',
 'modelV2',
 'navInstruction',
 'navModelDEPRECATED',
 'navThumbnail',
 'onroadEvents',
 'pandaStates',
 'peripheralState',
 'procLog',
 'qRoadEncodeIdx',
 'qcomGnss',
 'radarState',
 'roadCameraState',
 'roadEncodeIdx',
 'sendcan',
 'sentinel',
 'temperatureSensor',
 'thumbnail',
 'uiDebug',
 'uiPlan',
 'wideRoadCameraState',
 'wideRoadEncodeIdx'}

In [42]:
# These types were recorded in the TrondheimShow dataset rlog
trondheim_show_rlog_types = set([
    "accelerometer",
    "androidLog",
    "cameraOdometry",
    "can",
    "carControl",
    "carParams",
    "carState",
    "controlsState",
    "deviceState",
    "driverCameraState",
    "driverEncodeIdx",
    "driverMonitoringState",
    "driverStateV2",
    "gpsLocation",
    "gyroscope",
    "initData",
    "lateralPlanDEPRECATED",
    "liveCalibration",
    "liveLocationKalman",
    "liveParameters",
    "liveTorqueParameters",
    "liveTracks",
    "logMessage",
    "longitudinalPlan",
    "magnetometer",
    "managerState",
    "mapRenderState",
    "microphone",
    "modelV2",
    "navInstruction",
    "navModel",
    "navThumbnail",
    "onroadEvents",
    "pandaStates",
    "peripheralState",
    "procLog",
    "qRoadEncodeIdx",
    "qcomGnss",
    "radarState",
    "roadCameraState",
    "roadEncodeIdx",
    "sentinel",
    "temperatureSensor",
    "thumbnail",
    "uiDebug",
    "uiPlan",
    "wideRoadCameraState",
    "wideRoadEncodeIdx"
])
rlog_types - trondheim_show_rlog_types

{'clocks', 'navModelDEPRECATED', 'sendcan'}

In [43]:
trondheim_show_rlog_types - rlog_types

{'lateralPlanDEPRECATED', 'navModel'}

## Exploring the rlog data fields

In [44]:
# accelerometer
accelerometer = [log for log in rlog_list if log.which() == "accelerometer"]
accelerometer[0].accelerometer.to_dict()

{'acceleration': {'v': [9.781859397888184,
   0.2891986072063446,
   0.24608826637268066],
  'status': 1},
 'version': 1,
 'sensor': 1,
 'type': 1,
 'timestamp': 1588587765225,
 'source': 'lsm6ds3trc',
 'uncalibratedDEPRECATED': False}

In [45]:
# androidLog
androidLog = [log for log in rlog_list if log.which() == "androidLog"]
androidLog[0].androidLog.to_dict()

{'id': 0,
 'ts': 1709118955227168,
 'priority': 3,
 'pid': 0,
 'tid': 0,
 'message': '{"MESSAGE": "[thermald] engaged: False", "PRIORITY": "3", "SYSLOG_FACILITY": "1", "_BOOT_ID": "569d8999e5094a338e26c53853dc7811", "_HOSTNAME": "comma-c6a3e52b", "_MACHINE_ID": "ee405918b0d4d243a3e0a97365c57120", "_SOURCE_MONOTONIC_TIMESTAMP": "1588264471", "_TRANSPORT": "kernel"}'}

In [46]:
# cameraOdometry
cameraOdometry = [log for log in rlog_list if log.which() == "cameraOdometry"]
cameraOdometry[0].cameraOdometry.to_dict()

{'trans': [24.296321868896484, -0.03263568505644798, -0.019946783781051636],
 'rot': [-0.010202218778431416, 0.0023854849860072136, 0.02979431301355362],
 'transStd': [0.5363569855690002, 0.05314725637435913, 0.03263569995760918],
 'rotStd': [0.0031700232066214085,
  0.002227048622444272,
  0.0011528675677254796],
 'frameId': 26403,
 'timestampEof': 1588591242725,
 'wideFromDeviceEuler': [3.3248215913772583e-07,
  -0.02049574814736843,
  -0.01800425723195076],
 'wideFromDeviceEulerStd': [0.0012527357321232557,
  0.0028824410401284695,
  0.003801985876634717],
 'roadTransformTrans': [-4.0170561987906694e-08,
  1.3839962775819004e-07,
  1.3307054042816162],
 'roadTransformTransStd': [0.0009841345017775893,
  0.0009840067941695452,
  0.03989434614777565]}

In [47]:
# can
can = [log for log in rlog_list if log.which() == "can"]
can[0].to_dict()

{'can': [{'address': 583,
   'busTime': 0,
   'dat': b'\x01\x00\x03\xe8\x00\x00\x00\x00',
   'src': 1},
  {'address': 586,
   'busTime': 0,
   'dat': b'\x01\x00\x03\xe8\x00\x00\x00\x00',
   'src': 1},
  {'address': 320,
   'busTime': 0,
   'dat': b'r\x80\x00g \x00\x0c\x9e',
   'src': 128},
  {'address': 320, 'busTime': 0, 'dat': b'r\x80\x00g \x00\x0c\x9e', 'src': 0},
  {'address': 902,
   'busTime': 0,
   'dat': b'\xb0\xca\xad\xca\x99J\x99\n',
   'src': 0},
  {'address': 903,
   'busTime': 0,
   'dat': b'\xfb\xdd\x14\xea\x00\xdc\x06\x00',
   'src': 0},
  {'address': 589,
   'busTime': 0,
   'dat': b'\x01\x00\x03\xe8\x00\x00\x00\x00',
   'src': 1},
  {'address': 902,
   'busTime': 0,
   'dat': b'\xb0\xca\xad\xca\x99J\x99\n',
   'src': 128},
  {'address': 569,
   'busTime': 0,
   'dat': b'\x01}\x07\xd0}\x00\x00\x00',
   'src': 1},
  {'address': 903,
   'busTime': 0,
   'dat': b'\xfb\xdd\x14\xea\x00\xdc\x06\x00',
   'src': 128},
  {'address': 572,
   'busTime': 0,
   'dat': b'\x01}\x07\xd

In [48]:
# carControl
carControl = [log for log in rlog_list if log.which() == "carControl"]
carControl[0].carControl.to_dict()

{'enabled': False,
 'gasDEPRECATED': 0.0,
 'brakeDEPRECATED': 0.0,
 'steeringTorqueDEPRECATED': 0.0,
 'cruiseControl': {'cancel': False,
  'resume': False,
  'speedOverrideDEPRECATED': 0.0,
  'accelOverrideDEPRECATED': 0.0,
  'override': False},
 'hudControl': {'speedVisible': False,
  'setSpeed': 70.83333587646484,
  'lanesVisible': False,
  'leadVisible': False,
  'visualAlert': 'none',
  'audibleAlert': 'none',
  'rightLaneVisible': True,
  'leftLaneVisible': True,
  'rightLaneDepart': False,
  'leftLaneDepart': False,
  'leadDistanceBars': 0},
 'actuators': {'gas': 0.0,
  'brake': 0.0,
  'steer': -0.0,
  'steeringAngleDeg': 0.0,
  'accel': 0.0,
  'longControlState': 'off',
  'speed': 0.0,
  'curvature': 0.0012174933217465878,
  'steerOutputCan': 0.0},
 'activeDEPRECATED': False,
 'rollDEPRECATED': 0.0,
 'pitchDEPRECATED': 0.0,
 'actuatorsOutputDEPRECATED': {'gas': 0.0,
  'brake': 0.0,
  'steer': 0.0,
  'steeringAngleDeg': 0.0,
  'accel': 0.0,
  'longControlState': 'off',
  'speed':

In [49]:
# carParams
carParams = [log for log in rlog_list if log.which() == "carParams"]
carParams[0].carParams.to_dict()

{'carName': 'hyundai',
 'carFingerprint': 'KIA NIRO EV 2020',
 'enableGasInterceptorDEPRECATED': False,
 'pcmCruise': True,
 'enableCameraDEPRECATED': False,
 'enableDsu': False,
 'enableApgsDEPRECATED': False,
 'minEnableSpeed': -1.0,
 'minSteerSpeed': 0.0,
 'safetyModelDEPRECATED': 'silent',
 'safetyParamDEPRECATED': 0,
 'mass': 1743.076416015625,
 'wheelbase': 2.700000047683716,
 'centerToFront': 1.0800000429153442,
 'steerRatio': 13.600000381469727,
 'steerRatioRear': 0.0,
 'rotationalInertia': 2980.636962890625,
 'tireStiffnessFront': 88200.3203125,
 'tireStiffnessRear': 92951.15625,
 'longitudinalTuning': {'kpBP': [0.0],
  'kpV': [0.5],
  'kiBP': [0.0],
  'kiV': [0.0],
  'deadzoneBP': [0.0],
  'deadzoneV': [0.0],
  'kf': 1.0},
 'lateralTuning': {'torque': {'useSteeringAngle': True,
   'kp': 1.0,
   'ki': 0.10000000149011612,
   'friction': 0.09236802160739899,
   'kf': 1.0,
   'steeringAngleDeadzoneDeg': 0.0,
   'latAccelFactor': 2.921595573425293,
   'latAccelOffset': 0.0}},
 's

In [50]:
# carState
carState = [log for log in rlog_list if log.which() == "carState"]
carState[0].carState.to_dict()

{'vEgo': 23.633752822875977,
 'wheelSpeeds': {'fl': 23.72395896911621,
  'fr': 23.72395896911621,
  'rl': 23.55034637451172,
  'rr': 23.55034637451172},
 'gas': 0.16929133236408234,
 'gasPressed': True,
 'brake': 0.0,
 'brakePressed': False,
 'steeringAngleDeg': -2.0999999046325684,
 'steeringTorque': -57.0,
 'steeringPressed': False,
 'cruiseState': {'enabled': False,
  'speed': 8.333333015441895,
  'available': False,
  'speedOffset': 0.0,
  'standstill': False,
  'nonAdaptive': False,
  'speedCluster': 8.333333015441895},
 'events': [{'name': 'gasPressedOverride',
   'enable': False,
   'noEntry': False,
   'warning': False,
   'userDisable': False,
   'softDisable': False,
   'immediateDisable': False,
   'preEnable': False,
   'permanent': False,
   'overrideLongitudinal': True,
   'overrideLateral': False},
  {'name': 'wrongCarMode',
   'enable': False,
   'noEntry': True,
   'warning': False,
   'userDisable': True,
   'softDisable': False,
   'immediateDisable': False,
   'preE

In [51]:
# controlsState
controlsState = [log for log in rlog_list if log.which() == "controlsState"]
controlsState[0].controlsState.to_dict()

{'vEgoDEPRECATED': 0.0,
 'aEgoDEPRECATED': 0.0,
 'vPid': 23.633752822875977,
 'vTargetLead': 0.0,
 'upAccelCmd': 0.0,
 'uiAccelCmd': 0.0,
 'yActualDEPRECATED': 0.0,
 'yDesDEPRECATED': 0.0,
 'upSteerDEPRECATED': 0.0,
 'uiSteerDEPRECATED': 0.0,
 'aTargetMinDEPRECATED': 0.0,
 'aTargetMaxDEPRECATED': 0.0,
 'jerkFactorDEPRECATED': 0.0,
 'angleSteersDEPRECATED': 0.0,
 'hudLeadDEPRECATED': 0,
 'cumLagMs': 149.04257202148438,
 'canMonoTimeDEPRECATED': 0,
 'radarStateMonoTimeDEPRECATED': 0,
 'mdMonoTimeDEPRECATED': 0,
 'enabled': False,
 'steerOverrideDEPRECATED': False,
 'vCruise': 255.0,
 'rearViewCamDEPRECATED': False,
 'awarenessStatusDEPRECATED': 0.0,
 'angleModelBiasDEPRECATED': 0.0,
 'longitudinalPlanMonoTime': 1588533452256,
 'steeringAngleDesiredDegDEPRECATED': 0.0,
 'longControlState': 'off',
 'state': 'disabled',
 'vEgoRawDEPRECATED': 0.0,
 'ufAccelCmd': 0.0,
 'ufSteerDEPRECATED': 0.0,
 'aTarget': 0.0,
 'active': False,
 'curvature': 0.0011583779705688357,
 'alertStatus': 'normal',
 

In [52]:
# deviceState
deviceState = [log for log in rlog_list if log.which() == "deviceState"]
deviceState[0].deviceState.to_dict()

{'cpu0DEPRECATED': 0,
 'cpu1DEPRECATED': 0,
 'cpu2DEPRECATED': 0,
 'cpu3DEPRECATED': 0,
 'memDEPRECATED': 0,
 'gpuDEPRECATED': 0,
 'batDEPRECATED': 0,
 'freeSpacePercent': 80.45804595947266,
 'batteryPercentDEPRECATED': 0,
 'fanSpeedPercentDesired': 30,
 'started': True,
 'usbOnlineDEPRECATED': False,
 'startedMonoTime': 266901202969,
 'thermalStatus': 'green',
 'batteryCurrentDEPRECATED': 0,
 'batteryVoltageDEPRECATED': 0,
 'chargingErrorDEPRECATED': False,
 'chargingDisabledDEPRECATED': False,
 'memoryUsagePercent': 54,
 'cpuUsagePercentDEPRECATED': 0,
 'pa0DEPRECATED': 0,
 'networkType': 'none',
 'offroadPowerUsageUwh': 0,
 'networkStrength': 'unknown',
 'carBatteryCapacityUwh': 30006255,
 'cpuTempC': [68.5999984741211,
  67.5999984741211,
  69.30000305175781,
  68.5999984741211,
  68.9000015258789,
  66.5999984741211,
  66.30000305175781,
  67.30000305175781],
 'gpuTempC': [70.5999984741211, 68.0],
 'memoryTempC': 62.400001525878906,
 'batteryTempCDEPRECATED': 0.0,
 'ambientTempCDE

In [53]:
# driverCameraState
driverCameraState = [log for log in rlog_list if log.which() == "driverCameraState"]
driverCameraState[0].driverCameraState.to_dict()

{'frameId': 26402,
 'encodeId': 0,
 'timestampEof': 1588541462777,
 'frameLengthDEPRECATED': 0,
 'integLines': 180,
 'globalGainDEPRECATED': 0,
 'frameType': 'front',
 'timestampSof': 1588526639339,
 'lensPosDEPRECATED': 0,
 'lensSagDEPRECATED': 0.0,
 'lensErrDEPRECATED': 0.0,
 'lensTruePosDEPRECATED': 0.0,
 'gain': 8.692500114440918,
 'recoverStateDEPRECATED': 0,
 'highConversionGain': True,
 'measuredGreyFraction': 0.28125,
 'targetGreyFraction': 0.28492632508277893,
 'processingTime': 0.015036302618682384,
 'frameIdSensor': 0,
 'sensor': 'ox03c10',
 'exposureValPercent': 0.6865414977073669,
 'requestId': 26403}

In [54]:
# driverEncodeIdx
driverEncodeIdx = [log for log in rlog_list if log.which() == "driverEncodeIdx"]
driverEncodeIdx[0].driverEncodeIdx.to_dict()

{'frameId': 26402,
 'type': 'fullHEVC',
 'encodeId': 26400,
 'segmentNum': 22,
 'segmentId': 0,
 'segmentIdEncode': 0,
 'timestampSof': 1588526639339,
 'timestampEof': 1588541462777,
 'flags': 2147500040,
 'len': 231638}

In [55]:
# driverMonitoringState
driverMonitoringState = [log for log in rlog_list if log.which() == "driverMonitoringState"]
driverMonitoringState[6].driverMonitoringState.to_dict()

{'events': [],
 'faceDetected': True,
 'isDistracted': True,
 'awarenessStatus': 1.0,
 'isRHD': False,
 'rhdCheckedDEPRECATED': False,
 'posePitchOffset': -0.399808406829834,
 'posePitchValidCount': 7200,
 'poseYawOffset': 0.02051575481891632,
 'poseYawValidCount': 7200,
 'stepChange': 0.004545454401522875,
 'awarenessActive': 1.0,
 'awarenessPassive': 1.0,
 'isLowStd': True,
 'hiStdCount': 0,
 'isPreviewDEPRECATED': False,
 'isActiveMode': True,
 'distractedType': 1}

In [56]:
# driverStateV2
driverMonitoringState = [log for log in rlog_list if log.which() == "driverStateV2"]
driverMonitoringState[0].driverStateV2.to_dict()


{'frameId': 26402,
 'modelExecutionTime': 0.020834792405366898,
 'dspExecutionTime': 0.019548334181308746,
 'rawPredictions': b'',
 'poorVisionProb': 0.0009158340399153531,
 'wheelOnRightProb': 0.0009158340399153531,
 'leftDriverData': {'faceOrientation': [-0.16593405604362488,
   0.3446322977542877,
   0.0255283173173666],
  'faceOrientationStd': [0.06680338829755783,
   0.060318585485219955,
   0.04440271109342575],
  'facePosition': [0.12764158844947815, -0.07658495008945465],
  'facePositionStd': [0.0034572340082377195, 0.004029484931379557],
  'faceProb': 0.9483497738838196,
  'leftEyeProb': 0.7251346707344055,
  'rightEyeProb': 0.6714593172073364,
  'leftBlinkProb': 0.550879955291748,
  'rightBlinkProb': 0.5382177829742432,
  'sunglassesProb': 0.0009158340399153531,
  'occludedProb': 0.4872386157512665,
  'readyProb': [0.5634764432907104,
   0.4872386157512665,
   0.44912010431289673,
   0.43652352690696716],
  'notReadyProb': [0.07223648577928543, 0.5127613544464111]},
 'rightDr

In [57]:
# gpsLocation - removed in Orkdal set
# gpsLocation = [log for log in rlog_list if log.which() == "gpsLocation"]
# gpsLocation[0].gpsLocation.to_dict()

In [58]:
# gyroscope
gyroscope = [log for log in rlog_list if log.which() == "gyroscope"]
gyroscope[0].gyroscope.to_dict()

{'gyroUncalibrated': {'v': [-0.014508049935102463,
   -0.018478674814105034,
   0.0328340083360672],
  'status': 1},
 'version': 2,
 'sensor': 5,
 'type': 16,
 'timestamp': 1588578127880,
 'source': 'lsm6ds3trc',
 'uncalibratedDEPRECATED': False}

In [59]:
# initData
initData = [log for log in rlog_list if log.which() == "initData"]
initData[0].initData.to_dict()


{'kernelArgs': ['rcupdate.rcu_expedited=1',
  'console=ttyMSM0,115200n8',
  'quiet',
  'loglevel=3',
  'earlycon=msm_geni_serial,0xA84000',
  'androidboot.hardware=qcom',
  'androidboot.console=ttyMSM0',
  'video=DSI-1:1080x2160@60e',
  'ehci-hcd.park=3',
  'lpm_levels.sleep_disabled=1',
  'service_locator.enable=1',
  'androidboot.selinux=permissive',
  'firmware_class.path=/lib/firmware/updates',
  'net.ifnames=0',
  'dyndbg=""',
  'root=/dev/sda6',
  'androidboot.bootdevice=1d84000.ufshc',
  'androidboot.serialno=c6a3e52b',
  'androidboot.baseband=sda',
  'msm_drm.dsi_display0=dsi_fhd_ea8074_1080_cmd_display:',
  'androidboot.slot_suffix=_a',
  'skip_initramfs',
  'rootwait',
  'ro',
  'init=/sbin/init'],
 'dongleId': 'b1f2aa9141431844',
 'deviceType': 'tizi',
 'version': '0.9.7',
 'dirty': False,
 'gitCommit': '8e03cdfc2ae72ea09208ce41ddc8f7a118c3b973',
 'gitBranch': 'master',
 'passive': False,
 'gitRemote': 'https://github.com/commaai/openpilot.git',
 'kernelVersion': 'Linux vers

In [60]:
# lateralPlanDEPRECATED - removed in Orkdal set
# lateralPlanDEPRECATED = [log for log in rlog_list if log.which() == "lateralPlanDEPRECATED"]
# lateralPlanDEPRECATED[0].lateralPlanDEPRECATED.to_dict()

In [61]:
# liveCalibration
liveCalibration = [log for log in rlog_list if log.which() == "liveCalibration"]
liveCalibration[0].liveCalibration.to_dict()

{'calStatusDEPRECATED': 0,
 'calCycle': 0,
 'calPerc': 100,
 'rpyCalib': [-7.085435936460271e-05,
  0.06223521754145622,
  -0.009472996927797794],
 'rpyCalibSpread': [0.0008035845821723342,
  0.004331819247454405,
  0.012904700823128223],
 'validBlocks': 50,
 'wideFromDeviceEuler': [1.3453875453706132e-06,
  -0.02075459435582161,
  -0.020755073055624962],
 'calStatus': 'calibrated',
 'height': [1.2678545713424683]}

In [62]:
# liveLocationKalman
liveLocationKalman = [log for log in rlog_list if log.which() == "liveLocationKalman"]
liveLocationKalman[0].liveLocationKalman.to_dict()

{'positionECEF': {'value': [2827599.195378303,
   507626.7106143508,
   5675498.164355054],
  'std': [1.3295949442612172, 1.475196033369927, 1.2396405626574638],
  'valid': True},
 'positionGeodetic': {'value': [63.30729861188737,
   10.17764872883941,
   57.677240067808526],
  'std': [nan, nan, nan],
  'valid': True},
 'velocityECEF': {'value': [5.966951754288196,
   -23.26365942234526,
   -0.6252250181627274],
  'std': [0.18416800970335082, 0.2798021283262968, 0.166954397138811],
  'valid': True},
 'velocityNED': {'value': [-1.8552328373653153,
   -23.951974840348587,
   -0.2329842217790059],
  'std': [nan, nan, nan],
  'valid': True},
 'velocityDevice': {'value': [23.974494869691426,
   -0.24392669408496043,
   -1.5353712226250433],
  'std': [0.28112896501049534, 0.12312082404434391, 0.20658074965091372],
  'valid': True},
 'accelerationDevice': {'value': [0.19383310170215504,
   0.8490712480861001,
   -0.10349061579303069],
  'std': [0.3420853040853713, 0.34136685778725595, 0.33782

In [63]:
# liveParameters
liveParameters = [log for log in rlog_list if log.which() == "liveParameters"]
liveParameters[0].liveParameters.to_dict()

{'valid': True,
 'gyroBias': 0.0,
 'angleOffsetDeg': -0.034350939095020294,
 'angleOffsetAverageDeg': -0.21542784571647644,
 'stiffnessFactor': 1.0121945142745972,
 'steerRatio': 13.357989311218262,
 'sensorValid': True,
 'yawRateDEPRECATED': 0.0,
 'posenetSpeed': 0.0,
 'posenetValid': True,
 'angleOffsetFastStd': 0.0064011975191533566,
 'angleOffsetAverageStd': 0.0037123302463442087,
 'stiffnessFactorStd': 0.007465363945811987,
 'steerRatioStd': 0.10521427541971207,
 'roll': 0.07632264494895935}

In [64]:
# liveTorqueParameters
liveTorqueParameters = [log for log in rlog_list if log.which() == "liveTorqueParameters"]
liveTorqueParameters[0].liveTorqueParameters.to_dict()

{'liveValid': False,
 'latAccelFactorRaw': 0.0,
 'latAccelOffsetRaw': 0.0,
 'frictionCoefficientRaw': 0.0,
 'latAccelFactorFiltered': 2.921595573425293,
 'latAccelOffsetFiltered': 0.0,
 'frictionCoefficientFiltered': 0.09236802160739899,
 'totalBucketPoints': 0.0,
 'decay': 50.0,
 'maxResets': 1.0,
 'version': 1,
 'useParams': True}

In [65]:
# liveTracks
liveTracks = [log for log in rlog_list if log.which() == "liveTracks"]
liveTracks[5].liveTracks

<capnp list reader []>

In [66]:
# logMessage
logMessage = [log for log in rlog_list if log.which() == "logMessage"]
logMessage[0].logMessage

'{"created": 1709118955.5415688, "ctx": {"branch": "master", "commit": "8e03cdfc2ae72ea09208ce41ddc8f7a118c3b973", "daemon": "loggerd", "device": "tizi", "dirty": false, "dongle_id": "b1f2aa9141431844", "origin": "github.com/commaai/openpilot", "version": "0.9.7"}, "filename": "system/loggerd/loggerd.cc", "funcname": "handle_encoder_msg", "levelnum": 10, "lineno": 161, "msg": "rotate 21 -> 22 ready 4/4 for driverEncodeData"}'

In [67]:
# longitudinalPlan
longitudinalPlan = [log for log in rlog_list if log.which() == "longitudinalPlan"]
longitudinalPlan[0].longitudinalPlan.to_dict()


{'lateralValidDEPRECATED': False,
 'longitudinalValidDEPRECATED': False,
 'vTargetDEPRECATED': 0.0,
 'aTargetMinDEPRECATED': 0.0,
 'aTargetMaxDEPRECATED': 0.0,
 'jerkFactorDEPRECATED': 0.0,
 'hasLead': False,
 'fcw': False,
 'modelMonoTime': 1588578368037,
 'radarStateMonoTimeDEPRECATED': 0,
 'laneWidthDEPRECATED': 0.0,
 'vTargetFutureDEPRECATED': 0.0,
 'longitudinalPlanSource': 'cruise',
 'vCruiseDEPRECATED': 0.0,
 'aCruiseDEPRECATED': 0.0,
 'aTargetDEPRECATED': 0.0,
 'gpsPlannerActiveDEPRECATED': False,
 'vMaxDEPRECATED': 0.0,
 'vCurvatureDEPRECATED': 0.0,
 'decelForTurnDEPRECATED': False,
 'hasLeftLaneDEPRECATED': False,
 'hasRightLaneDEPRECATED': False,
 'mapValidDEPRECATED': False,
 'vStartDEPRECATED': 0.0,
 'aStartDEPRECATED': 0.0,
 'radarValidDEPRECATED': False,
 'processingDelay': -1588578353152.0,
 'radarCanErrorDEPRECATED': False,
 'commIssueDEPRECATED': False,
 'accels': [0.12828229367733002,
  0.13890574872493744,
  0.17077608406543732,
  0.22163313627243042,
  0.2876213192

In [68]:
# magnetometer
magnetometer = [log for log in rlog_list if log.which() == "magnetometer"]
magnetometer[0].magnetometer.to_dict()

{'magneticUncalibrated': {'v': [-0.24505615234375,
   -0.001708984375,
   0.59185791015625,
   -0.05828857421875,
   -0.1959228515625,
   1.38018798828125],
  'status': 1},
 'version': 1,
 'sensor': 3,
 'type': 14,
 'timestamp': 1588568394287,
 'source': 'mmc5603nj',
 'uncalibratedDEPRECATED': False}

In [69]:
# managerState
managerState = [log for log in rlog_list if log.which() == "managerState"]
managerState[0].managerState.to_dict()

{'processes': [{'name': 'manage_athenad',
   'pid': 0,
   'running': False,
   'exitCode': 0,
   'shouldBeRunning': False},
  {'name': 'camerad',
   'pid': 51324,
   'running': True,
   'exitCode': 0,
   'shouldBeRunning': True},
  {'name': 'logcatd',
   'pid': 51325,
   'running': True,
   'exitCode': 0,
   'shouldBeRunning': True},
  {'name': 'proclogd',
   'pid': 51329,
   'running': True,
   'exitCode': 0,
   'shouldBeRunning': True},
  {'name': 'logmessaged',
   'pid': 50263,
   'running': True,
   'exitCode': 0,
   'shouldBeRunning': True},
  {'name': 'micd',
   'pid': 51332,
   'running': True,
   'exitCode': 0,
   'shouldBeRunning': True},
  {'name': 'timed',
   'pid': 50264,
   'running': True,
   'exitCode': 0,
   'shouldBeRunning': True},
  {'name': 'dmonitoringmodeld',
   'pid': 51333,
   'running': True,
   'exitCode': 0,
   'shouldBeRunning': True},
  {'name': 'encoderd',
   'pid': 51335,
   'running': True,
   'exitCode': 0,
   'shouldBeRunning': True},
  {'name': 'strea

In [70]:
# mapRenderState
mapRenderState = [log for log in rlog_list if log.which() == "mapRenderState"]
mapRenderState[0].mapRenderState.to_dict()

{'locationMonoTime': 1588830924964, 'renderTime': 0.0, 'frameId': 2638}

In [71]:
# microphone
microphone = [log for log in rlog_list if log.which() == "microphone"]
microphone[0].microphone.to_dict()

{'soundPressure': 0.0715392455458641,
 'soundPressureWeightedDb': 51.051551818847656,
 'filteredSoundPressureWeightedDb': 0.0,
 'soundPressureWeighted': 0.007138511631637812}

In [72]:
# modelV2
modelV2 = [log for log in rlog_list if log.which() == "modelV2"]
modelV2[0].modelV2.to_dict()


{'frameId': 26403,
 'frameAge': 0,
 'frameDropPerc': 0.0,
 'timestampEof': 1588591242725,
 'position': {'x': [-0.7911596298217773,
   -1.0568023920059204,
   -0.08450406789779663,
   1.082215428352356,
   2.7269692420959473,
   4.830905437469482,
   7.420556545257568,
   10.519880294799805,
   14.06656551361084,
   18.053741455078125,
   22.534658432006836,
   27.48931121826172,
   32.94861602783203,
   38.78273010253906,
   45.179805755615234,
   51.977134704589844,
   59.27267837524414,
   66.99696350097656,
   75.11167907714844,
   83.591796875,
   92.85498046875,
   102.40267181396484,
   112.60613250732422,
   122.74519348144531,
   133.80381774902344,
   145.046875,
   156.55142211914062,
   168.72525024414062,
   181.37628173828125,
   193.60574340820312,
   207.1272735595703,
   220.69248962402344,
   234.6352081298828],
  'y': [0.22902444005012512,
   0.22790850698947906,
   0.2305973768234253,
   0.2324514240026474,
   0.22782456874847412,
   0.23194058239459991,
   0.2391077

In [73]:
# navInstruction
navInstruction = [log for log in rlog_list if log.which() == "navInstruction"]
navInstruction[0].navInstruction.to_dict()

{'maneuverDistance': 0.0,
 'distanceRemaining': 0.0,
 'timeRemaining': 0.0,
 'timeRemainingTypical': 0.0,
 'showFull': False,
 'speedLimit': 0.0,
 'speedLimitSign': 'mutcd'}

In [74]:
# navModel
navModel = [log for log in rlog_list if log.which() == "navModel"]
navModel[0].navModel.to_dict()

IndexError: list index out of range

In [ ]:
# navThumbnail
navThumbnail = [log for log in rlog_list if log.which() == "navThumbnail"]
navThumbnail[0].navThumbnail.to_dict()

{'frameId': 1200,
 'timestampEof': 4844012664413,
 'thumbnail': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00d\x00d\x00\x00\xff\xdb\x00C\x00\x10\x0b\x0c\x0e\x0c\n\x10\x0e\r\x0e\x12\x11\x10\x13\x18(\x1a\x18\x16\x16\x181#%\x1d(:3=<9387@H\\N@DWE78PmQW_bghg>Mqypdx\\egc\xff\xdb\x00C\x01\x11\x12\x12\x18\x15\x18/\x1a\x1a/cB8Bcccccccccccccccccccccccccccccccccccccccccccccccccc\xff\xc0\x00\x11\x08\x01\x00\x01\x00\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x99\x9a\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa9\xaa\xb2\xb3\xb4\xb5\xb6\xb7\xb8\xb9\xba\xc2\xc3\xc4\xc5\xc6\xc7\xc8\x

In [ ]:
# onroadEvents
onroadEvents = [log for log in rlog_list if log.which() == "onroadEvents"]
onroadEvents[0].onroadEvents[0].to_dict()

{'name': 'doorOpen',
 'enable': False,
 'noEntry': True,
 'warning': False,
 'userDisable': False,
 'softDisable': True,
 'immediateDisable': False,
 'preEnable': False,
 'permanent': False,
 'overrideLongitudinal': False,
 'overrideLateral': False}

In [ ]:
# pandaStates
pandaStates = [log for log in rlog_list if log.which() == "pandaStates"]
pandaStates[0].pandaStates[0].to_dict()

{'voltage': 13783,
 'current': 0,
 'ignitionLine': True,
 'controlsAllowed': False,
 'gasInterceptorDetectedDEPRECATED': False,
 'startedSignalDetectedDEPRECATED': False,
 'hasGpsDEPRECATED': False,
 'rxBufferOverflow': 0,
 'txBufferOverflow': 0,
 'gmlanSendErrs': 0,
 'pandaType': 'tres',
 'fanSpeedRpmDEPRECATED': 0,
 'usbPowerModeDEPRECATED': 'none',
 'ignitionCan': False,
 'safetyModel': 'hyundai',
 'faultStatus': 'none',
 'powerSaveEnabled': False,
 'uptime': 4812,
 'faults': [],
 'safetyRxInvalid': 0,
 'safetyParamDEPRECATED': 0,
 'harnessStatus': 'flipped',
 'heartbeatLost': False,
 'alternativeExperience': 1,
 'safetyTxBlocked': 0,
 'interruptLoad': 0.1484825313091278,
 'safetyParam2DEPRECATED': 0,
 'safetyParam': 1,
 'fanPower': 29,
 'canState0': {'busOff': False,
  'busOffCnt': 169,
  'errorWarning': False,
  'errorPassive': False,
  'lastError': 'noError',
  'lastStoredError': 'formError',
  'lastDataError': 'noChange',
  'lastDataStoredError': 'noError',
  'receiveErrorCnt': 

In [ ]:
# peripheralState
peripheralState = [log for log in rlog_list if log.which() == "peripheralState"]
peripheralState[0].peripheralState.to_dict()

{'pandaType': 'tres',
 'voltage': 14081,
 'current': 729,
 'fanSpeedRpm': 2185,
 'usbPowerModeDEPRECATED': 'none'}

In [ ]:
# procLog
procLog = [log for log in rlog_list if log.which() == "procLog"]
procLog[0].procLog.to_dict()

{'cpuTimes': [{'cpuNum': 0,
   'user': 899.5800170898438,
   'nice': 0.05999999865889549,
   'system': 687.9600219726562,
   'idle': 2813.050048828125,
   'iowait': 9.779999732971191,
   'irq': 65.23999786376953,
   'softirq': 28.350000381469727},
  {'cpuNum': 1,
   'user': 791.6599731445312,
   'nice': 0.05999999865889549,
   'system': 758.6400146484375,
   'idle': 2706.31005859375,
   'iowait': 11.069999694824219,
   'irq': 214.89999389648438,
   'softirq': 16.0},
  {'cpuNum': 2,
   'user': 798.3499755859375,
   'nice': 0.05000000074505806,
   'system': 455.3500061035156,
   'idle': 3332.840087890625,
   'iowait': 9.0,
   'irq': 23.75,
   'softirq': 11.930000305175781},
  {'cpuNum': 3,
   'user': 593.1500244140625,
   'nice': 0.05999999865889549,
   'system': 886.3099975585938,
   'idle': 3109.239990234375,
   'iowait': 5.980000019073486,
   'irq': 19.43000030517578,
   'softirq': 12.430000305175781},
  {'cpuNum': 4,
   'user': 1543.719970703125,
   'nice': 0.009999999776482582,
   '

In [ ]:
# qRoadEncodeIdx
qRoadEncodeIdx = [log for log in rlog_list if log.which() == "qRoadEncodeIdx"]
qRoadEncodeIdx[0].qRoadEncodeIdx.to_dict()

{'frameId': 12002,
 'type': 'qcameraH264',
 'encodeId': 12000,
 'segmentNum': 10,
 'segmentId': 0,
 'segmentIdEncode': 0,
 'timestampSof': 4842250762851,
 'timestampEof': 4842265460872,
 'flags': 2147500040,
 'len': 2160}

In [ ]:
# qcomGnss
qcomGnss = [log for log in rlog_list if log.which() == "qcomGnss"]
qcomGnss[0].qcomGnss.to_dict()

{'drMeasurementReport': {'reason': 0,
  'seqNum': 1,
  'seqMax': 5,
  'rfLoss': 30,
  'systemRtcValid': True,
  'fCount': 4705958,
  'clockResets': 11,
  'systemRtcTime': 4831595,
  'gpsLeapSeconds': 18,
  'gpsLeapSecondsUncertainty': 0,
  'gpsToGlonassTimeBiasMilliseconds': 0.005850543268024921,
  'gpsToGlonassTimeBiasMillisecondsUncertainty': 1.4036235370440409e-05,
  'gpsWeek': 0,
  'gpsMilliseconds': 0,
  'gpsTimeBiasMs': 0,
  'gpsClockTimeUncertaintyMs': 0,
  'gpsClockSource': 0,
  'glonassClockSource': 10,
  'glonassYear': 8,
  'glonassDay': 59,
  'glonassMilliseconds': 54608604,
  'glonassTimeBias': -0.1258041113615036,
  'glonassClockTimeUncertainty': 15369259.0,
  'clockFrequencyBias': -0.5855321288108826,
  'clockFrequencyUncertainty': 87.64002227783203,
  'frequencySource': 2,
  'source': 'glonass',
  'sv': [{'svId': 255,
    'glonassFrequencyIndex': -6,
    'observationState': 'track',
    'observations': 154,
    'goodObservations': 154,
    'filterStages': 1,
    'predete

In [ ]:
# radarState
radarState = [log for log in rlog_list if log.which() == "radarState"]
radarState[0].radarState.to_dict()


{'angleOffsetDEPRECATED': 0.0,
 'calStatusDEPRECATED': 0,
 'leadOne': {'dRel': 0.0,
  'yRel': 0.0,
  'vRel': 0.0,
  'aRel': 0.0,
  'vLead': 0.0,
  'aLeadDEPRECATED': 0.0,
  'dPath': 0.0,
  'vLat': 0.0,
  'vLeadK': 0.0,
  'aLeadK': 0.0,
  'fcw': False,
  'status': False,
  'aLeadTau': 0.0,
  'modelProb': 0.0,
  'radar': False,
  'radarTrackId': -1},
 'leadTwo': {'dRel': 0.0,
  'yRel': 0.0,
  'vRel': 0.0,
  'aRel': 0.0,
  'vLead': 0.0,
  'aLeadDEPRECATED': 0.0,
  'dPath': 0.0,
  'vLat': 0.0,
  'vLeadK': 0.0,
  'aLeadK': 0.0,
  'fcw': False,
  'status': False,
  'aLeadTau': 0.0,
  'modelProb': 0.0,
  'radar': False,
  'radarTrackId': -1},
 'cumLagMs': -7.410253047943115,
 'mdMonoTime': 4842302050611,
 'ftMonoTimeDEPRECATED': 0,
 'calCycleDEPRECATED': 0,
 'calPercDEPRECATED': 0,
 'carStateMonoTime': 4842344071184,
 'radarErrors': []}

In [ ]:
# roadCameraState
roadCameraState = [log for log in rlog_list if log.which() == "roadCameraState"]
roadCameraState[0].roadCameraState.to_dict()

{'frameId': 12003,
 'encodeId': 0,
 'timestampEof': 4842315466601,
 'frameLengthDEPRECATED': 0,
 'integLines': 2,
 'globalGainDEPRECATED': 0,
 'frameType': 'unknown',
 'timestampSof': 4842300760403,
 'lensPosDEPRECATED': 0,
 'lensSagDEPRECATED': 0.0,
 'lensErrDEPRECATED': 0.0,
 'lensTruePosDEPRECATED': 0.0,
 'gain': 7.320000171661377,
 'recoverStateDEPRECATED': 0,
 'highConversionGain': True,
 'measuredGreyFraction': 0.28125,
 'targetGreyFraction': 0.3952854573726654,
 'processingTime': 0.00591348996385932,
 'frameIdSensor': 0,
 'sensor': 'ox03c10',
 'exposureValPercent': 0.005088916979730129,
 'requestId': 12004}

In [ ]:
# roadEncodeIdx
roadEncodeIdx = [log for log in rlog_list if log.which() == "roadEncodeIdx"]
roadEncodeIdx[0].roadEncodeIdx.to_dict()

{'frameId': 12002,
 'type': 'fullHEVC',
 'encodeId': 12000,
 'segmentNum': 10,
 'segmentId': 0,
 'segmentIdEncode': 0,
 'timestampSof': 4842250762851,
 'timestampEof': 4842265460872,
 'flags': 2147500040,
 'len': 131739}

In [ ]:
# sentinel
sentinel = [log for log in rlog_list if log.which() == "sentinel"]
sentinel[0].sentinel.to_dict()

{'type': 'startOfSegment', 'signal': 0}

In [ ]:
# temperatureSensor
temperatureSensor = [log for log in rlog_list if log.which() == "temperatureSensor"]
temperatureSensor[0].temperatureSensor.to_dict()

{'temperature': 57.2109375,
 'version': 1,
 'sensor': 0,
 'type': 13,
 'timestamp': 4842465625299,
 'source': 'lsm6ds3trc',
 'uncalibratedDEPRECATED': False}

In [ ]:
# thumbnail
thumbnail = [log for log in rlog_list if log.which() == "thumbnail"]
thumbnail[0].thumbnail.to_dict()


{'frameId': 12003,
 'timestampEof': 4842315466601,
 'thumbnail': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x10\x0b\x0c\x0e\x0c\n\x10\x0e\r\x0e\x12\x11\x10\x13\x18(\x1a\x18\x16\x16\x181#%\x1d(:3=<9387@H\\N@DWE78PmQW_bghg>Mqypdx\\egc\xff\xdb\x00C\x01\x11\x12\x12\x18\x15\x18/\x1a\x1a/cB8Bcccccccccccccccccccccccccccccccccccccccccccccccccc\xff\xc0\x00\x11\x08\x01.\x01\xe2\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x99\x9a\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa9\xaa\xb2\xb3\xb4\xb5\xb6\xb7\xb8\xb9\xba\xc2\xc3\xc4\xc5\xc6\xc7\x

In [ ]:
# uiDebug
uiDebug = [log for log in rlog_list if log.which() == "uiDebug"]
uiDebug[0].uiDebug.to_dict()

{'drawTimeMillis': 3.2723441123962402}

In [ ]:
# uiPlan
uiPlan = [log for log in rlog_list if log.which() == "uiPlan"]
uiPlan[0].uiPlan.to_dict()

{'position': {'x': [-0.6959930062294006,
   -1.082137942314148,
   -0.7976922988891602,
   -0.860908031463623,
   -0.8807353973388672,
   -0.864555835723877,
   -0.8652877807617188,
   -0.831049919128418,
   -0.9601631164550781,
   -0.8741931915283203,
   -0.9478302001953125,
   -0.896697998046875,
   -0.9429817199707031,
   -0.8727340698242188,
   -0.8934860229492188,
   -0.9415435791015625,
   -0.7681503295898438,
   -0.9997978210449219,
   -0.831146240234375,
   -0.7726593017578125,
   -0.559814453125,
   -0.5723609924316406,
   -0.42169952392578125,
   -0.43216705322265625,
   -0.5300216674804688,
   -0.612274169921875,
   -0.9468307495117188,
   -0.26416778564453125,
   -0.3587493896484375,
   -0.37115478515625,
   0.5387649536132812,
   0.8720779418945312,
   0.399017333984375],
  'y': [-0.007234580814838409,
   -0.004108160734176636,
   -0.003938598558306694,
   -0.007995830848813057,
   -0.009165717288851738,
   -0.011359862983226776,
   -0.013408507220447063,
   -0.00823305733

In [ ]:
# wideRoadCameraState
wideRoadCameraState = [log for log in rlog_list if log.which() == "wideRoadCameraState"]
wideRoadCameraState[0].wideRoadCameraState.to_dict()

{'frameId': 12003,
 'encodeId': 0,
 'timestampEof': 4842315605872,
 'frameLengthDEPRECATED': 0,
 'integLines': 60,
 'globalGainDEPRECATED': 0,
 'frameType': 'unknown',
 'timestampSof': 4842300826601,
 'lensPosDEPRECATED': 0,
 'lensSagDEPRECATED': 0.0,
 'lensErrDEPRECATED': 0.0,
 'lensTruePosDEPRECATED': 0.0,
 'gain': 7.320000171661377,
 'recoverStateDEPRECATED': 0,
 'highConversionGain': True,
 'measuredGreyFraction': 0.33984375,
 'targetGreyFraction': 0.3420105576515198,
 'processingTime': 0.010954635217785835,
 'frameIdSensor': 0,
 'sensor': 'ox03c10',
 'exposureValPercent': 0.19070322811603546,
 'requestId': 12004}

In [ ]:
# wideRoadEncodeIdx
wideRoadEncodeIdx = [log for log in rlog_list if log.which() == "wideRoadEncodeIdx"]
wideRoadEncodeIdx[0].wideRoadEncodeIdx.to_dict()

{'frameId': 12002,
 'type': 'fullHEVC',
 'encodeId': 12000,
 'segmentNum': 10,
 'segmentId': 0,
 'segmentIdEncode': 0,
 'timestampSof': 4842250864101,
 'timestampEof': 4842265603007,
 'flags': 2147500040,
 'len': 94696}

In [ ]:
# clocks
clocks = [log for log in rlog_list if log.which() == "clocks"]
clocks[0].clocks.to_dict()

{'bootTimeNanosDEPRECATED': 0,
 'monotonicNanosDEPRECATED': 0,
 'monotonicRawNanosDEPRECATD': 0,
 'wallTimeNanos': 1709122208580244358,
 'modemUptimeMillisDEPRECATED': 0}

In [ ]:
# sendcan
sendcan = [log for log in rlog_list if log.which() == "sendcan"]
sendcan[0].to_dict()

{'sendcan': [{'address': 832,
   'busTime': 0,
   'dat': b'\x13\x00\x00$\x94\x00\xdc\x11',
   'src': 0}],
 'logMonoTime': 4842303794882,
 'valid': True}

In [ ]:
# errorLogMessage
errorLogMessage = [log for log in rlog_list if log.which() == "errorLogMessage"]
errorLogMessage[0].errorLogMessage

'{"msg": "Failed to download assistance file", "ctx": {"daemon": "qcomgpsd", "dongle_id": "b1f2aa9141431844", "version": "0.9.7", "origin": "github.com/commaai/openpilot", "branch": "master", "commit": "8e03cdfc2ae72ea09208ce41ddc8f7a118c3b973", "dirty": false, "device": "tizi"}, "exc_info": "Traceback (most recent call last):\\n  File \\"/usr/local/pyenv/versions/3.11.4/lib/python3.11/site-packages/urllib3/connection.py\\", line 198, in _new_conn\\n    sock = connection.create_connection(\\n           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\\n  File \\"/usr/local/pyenv/versions/3.11.4/lib/python3.11/site-packages/urllib3/util/connection.py\\", line 60, in create_connection\\n    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):\\n               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\\n  File \\"/usr/local/pyenv/versions/3.11.4/lib/python3.11/socket.py\\", line 962, in getaddrinfo\\n    for res in _socket.getaddrinfo(host, port, family, type, proto, flags

# NEW

In [ ]:
# Huh, the rlog.json and qlog.json contains NaN values, but the loading doesn't crash because of it...
with open("rlog.json", "r") as f:
    rlog_json = json.load(f)
rlog_json

FileNotFoundError: [Errno 2] No such file or directory: 'rlog.json'

# The difference between rlog and qlog

I am not entirely sure, but ChatGPT claims rlog is the Raw Logs, while the qlog is the quantized logs.
This means that qlog is a simplified version of the raw data. 

This is likely true as the original qlog file is much smaller than the rlog file. The json file for qlog is 20.8 MB while rlog.json is over 200 MB.